# Analysis of disagreements

In this series of experiments, we use ablation studies to investigate the role of components highlighted by either Integrated Gradients or activation patching, but not both.

In [ ]:
# Set up

%load_ext autoreload
%autoreload 2

import torch
import random
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device, get_act_name
import numpy as np
import matplotlib.pyplot as plt

from attribution_methods import integrated_gradients, activation_patching, highlight_components
from testing import Task, TaskDataset, logit_diff_metric, identify_outliers, average_correlation, measure_overlap, test_single_ablated_performance, test_multi_ablated_performance
from plotting import plot_attn, plot_attn_comparison, plot_correlation, plot_correlation_comparison, plot_bar_chart

In [ ]:
torch.set_grad_enabled(False)
torch.cuda.empty_cache()

device = get_device()
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

In [ ]:
# Measure baseline performance

test_dataset = TaskDataset(Task.IOI)
test_dataloader = test_dataset.to_dataloader(batch_size=10)

baseline_scores = []

for i, (clean_input, _, labels) in enumerate(test_dataloader):
    clean_tokens = model.to_tokens(clean_input)
    logits = model(clean_tokens)
    baseline_scores.append(logit_diff_metric(logits, labels))

baseline_performance = sum(baseline_scores) / len(baseline_scores)

In [ ]:
ig_mlp = torch.load("results/aligned/ioi/ig_mlp.pt")
ig_attn = torch.load("results/aligned/ioi/ig_attn.pt")

ap_mlp = torch.load("results/aligned/ioi/ap_mlp.pt")
ap_attn = torch.load("results/aligned/ioi/ap_attn.pt")

## Identify disagreements

In [ ]:
# Identify statistically significant outlier components

scaled_ig_attn = ig_attn * 1e5
attn_outliers = identify_outliers(scaled_ig_attn, ap_attn)
mlp_outliers = identify_outliers(ig_mlp, ap_mlp)

In [ ]:
# Get the mean activations over a corrupt dataset

attn_outlier_hooks = [get_act_name("result", layer_idx) for layer_idx, _ in attn_outliers]
mlp_outlier_hooks = [get_act_name("post", layer_idx) for layer_idx, _ in mlp_outliers]

random_prompts = random.sample(test_dataset, 100)
prompts_tokens = model.to_tokens([p for p, _ in random_prompts])
_, prompt_cache = model.run_with_cache(
    prompts_tokens, 
    names_filter=lambda x: x in attn_outlier_hooks or x in mlp_outlier_hooks
)

mean_corrupt_activations = {}
for key in prompt_cache.keys():
    mean_values_over_prompts = torch.mean(prompt_cache[key], dim=0)
    mean_corrupt_activations[key] = torch.mean(mean_values_over_prompts, dim=0)

## Performance under isolated ablation

- Hypothesis: components which have high attribution scores in only one of the two methods are still important.
- Method: we measure the performance of a GPT2-small model when the components which are exclusive to only one method are ablated (with mean corrupt activations), one at a time.

### Attention heads

In [ ]:
attn_outlier_isolated_ablation_scores = dict()

for layer, idx in attn_outliers:
    score = test_single_ablated_performance(model, layer, idx, mean_corrupt_activations, Task.IOI, is_attn=True)
    attn_outlier_isolated_ablation_scores[(layer, idx)] = score

In [ ]:
plt.title("Model performance after isolated ablation of attention head outliers")
plt.xlabel("Ablated attention head position")
plt.ylabel("Model performance on IOI task")

xs = ["None"] + [str(t) for t in attn_outlier_isolated_ablation_scores.keys()]
ys = [baseline_performance] + attn_outlier_isolated_ablation_scores.values()

plt.bar(xs, ys)
plt.show()

### Neurons

In [ ]:
mlp_outlier_isolated_ablation_scores = dict()

for layer, idx in mlp_outliers:
    score = test_single_ablated_performance(model, layer, idx, mean_corrupt_activations, Task.IOI, is_attn=False)
    mlp_outlier_isolated_ablation_scores[(layer, idx)] = score

In [ ]:
plt.title("Model performance after isolated ablation of neuron outliers")
plt.xlabel("Ablated neuron position")
plt.ylabel("Model performance on IOI task")

xs = ["None"] + [str(t) for t in mlp_outlier_isolated_ablation_scores.keys()]
ys = [baseline_performance] + mlp_outlier_isolated_ablation_scores.values()

plt.bar(xs, ys)
plt.show()

## Performance under simultaneous ablation

- Hypothesis: components which have high attribution scores in only one of the two methods may be involved in multi-component interactions.
- Method: we measure the performance when the components exclusive to only one method are ablated all at once.

In [ ]:
# Ablate method-exclusive components

outlier_attn_multi_ablated_performance = test_multi_ablated_performance(model, attn_outliers, mean_corrupt_activations, Task.IOI, is_attn=True)

In [ ]:
# Ablate all IG-highlighted components
ig_attn_highlighted, ig_attn_indices = highlight_components(ig_attn)
ig_attn_multi_ablated_performance = test_multi_ablated_performance(model, ig_attn_indices, mean_corrupt_activations, Task.IOI, is_attn=True)

# Ablate all AP-highlighted components
ap_attn_highlighted, ap_attn_indices = highlight_components(ap_attn)
ap_attn_multi_ablated_performance = test_multi_ablated_performance(model, ap_attn_indices, mean_corrupt_activations, Task.IOI, is_attn=True)

# Ablate all IG and AP highlighted components
shared_attn_indices = (ig_attn_highlighted & ap_attn_highlighted).nonzero()
shared_attn_multi_ablated_performance = test_multi_ablated_performance(model, shared_attn_indices, mean_corrupt_activations, Task.IOI, is_attn=True)

In [ ]:
plt.title("Model performance after simultaneous ablation of attention heads")
plt.xlabel("Method(s) to highlight ablated attention heads")
plt.ylabel("Model performance on IOI task")

xs = ["None", "IG", "AP", "IG + AP", "Exclusive" ]
ys = [baseline_performance, ig_attn_multi_ablated_performance, ap_attn_multi_ablated_performance, shared_attn_multi_ablated_performance, outlier_attn_multi_ablated_performance]

plt.bar(xs, ys)
plt.show()

### Neurons

In [ ]:
# Ablate method-exclusive components
outlier_mlp_multi_ablated_performance = test_multi_ablated_performance(model, mlp_outliers, mean_corrupt_activations, Task.IOI, is_attn=False)

In [ ]:
# Ablate all IG-highlighted components
ig_mlp_highlighted, ig_mlp_indices = highlight_components(ig_mlp)
ig_mlp_multi_ablated_performance = test_multi_ablated_performance(model, ig_mlp_indices, mean_corrupt_activations, Task.IOI, is_attn=False)

# Ablate all AP-highlighted components
ap_mlp_highlighted, ap_mlp_indices = highlight_components(ap_mlp)
ap_mlp_multi_ablated_performance = test_multi_ablated_performance(model, ap_mlp_indices, mean_corrupt_activations, Task.IOI, is_attn=False)

# Ablate all IG and AP highlighted components
shared_mlp_indices = (ig_mlp_highlighted & ap_mlp_highlighted).nonzero()
shared_mlp_multi_ablated_performance = test_multi_ablated_performance(model, shared_mlp_indices, mean_corrupt_activations, Task.IOI, is_attn=False)

In [ ]:
plt.title("Model performance after simultaneous ablation of neurons")
plt.xlabel("Method(s) to highlight ablated neurons")
plt.ylabel("Model performance on IOI task")

xs = ["None", "IG", "AP", "IG + AP", "Exclusive" ]
ys = [baseline_performance, ig_mlp_multi_ablated_performance, ap_mlp_multi_ablated_performance, shared_mlp_multi_ablated_performance, outlier_mlp_multi_ablated_performance]

plt.bar(xs, ys)
plt.show()